In [1]:
import sys
sys.path.append('../')
import transport_frames.src.graph_builder.criteria as criteria
import transport_frames.src.metrics.indicators as indicators
import transport_frames.src.metrics.grade_territory as grade_territory
import transport_frames.src.graph_builder.graphbuilder as graphbuilder
import momepy
import osmnx as ox
import geopandas as gpd
import shapely
import pandas as pd
import networkx as nx
import numpy as np
import json
from blocksnet import  AdjacencyCalculator

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
region_name = 'Тульская_область'
local_crs = 32637

In [3]:

PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # set where are no services 

polygons188 = gpd.read_file(f'../data/{region_name}/settlement.geojson')
polygons18 = gpd.read_file(f'../data/{region_name}/district.geojson')

points = gpd.read_file(f'../data/{region_name}/{region_name}_region_points.geojson')
# points188 = shapely.centroid(polygons188.geometry).set_crs(polygons188.crs)
points18 = points[points['is_admin_centre_district']==True]


railway_stops = gpd.read_file(f'../data/{region_name}/railway_station.geojson')

fuel = gpd.read_file(f'../data/{region_name}/Копия fuel.geojson')

# ports = gpd.read_file(f'../data/{region_name}/ferry_terminal.geojson')
ports = PLACEHOLDER

local_aero = gpd.read_file(f'../data/{region_name}/aerodrome.geojson')

international_aero = local_aero[local_aero['aerodrome:type']=='international']
international_aero['geometry'] = shapely.centroid(international_aero['geometry']).set_crs(international_aero.crs)

local_aero = local_aero[local_aero['aerodrome:type']!='international']
local_aero['geometry'] = shapely.centroid(local_aero['geometry']).set_crs(local_aero.crs)


water_objects = gpd.read_file(f'../data/{region_name}/Копия water.geojson')

oopt = gpd.read_file(f'../data/{region_name}/Копия nature_reserve.geojson')

train_paths = gpd.read_file(f'../data/{region_name}/railway.geojson')

bus_stops = gpd.read_file(f'../data/{region_name}/bus_stop.geojson')
####
neud = polygons18.iloc[[3]].to_crs(local_crs)
neud_center = neud.geometry.representative_point()
neud_center = gpd.GeoDataFrame([{'geometry': neud_center.iloc[0]}], crs=neud.crs).to_crs(local_crs)



In [4]:
region_capital = ox.geocode_to_gdf('N34389350',by_osmid=True)
city = ox.geocode_to_gdf(f'R{81993}', by_osmid=True).to_crs(epsg=4326)

In [5]:
citygraph = nx.read_graphml(f'/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_uds.graphml')
citygraph = graphbuilder.convert_list_attr_from_str(citygraph)
citygraph = indicators.prepare_graph(citygraph)

In [6]:
inter = nx.read_graphml(f"../preprocessed/{region_name}/{region_name}_inter.graphml")
inter = graphbuilder.convert_list_attr_from_str(inter)
inter = indicators.prepare_graph(inter)
bus_routes = gpd.read_file(f'../preprocessed/{region_name}/{region_name}_bus_routes.geojson')


In [7]:
citygraph_n,citygraph_e = momepy.nx_to_gdf(citygraph)

In [8]:
services = {
    'railway_stops' : railway_stops,
    'fuel' : fuel,
    'ports' : ports,
    'local_aero' : local_aero,
    'international_aero' : international_aero,
    'oopt' : oopt,
    'water_objects' : water_objects,
    'train_paths' : train_paths,
    'bus_stops' : bus_stops,
    'capital':region_capital,
    'points':points,
    'bus_routes' : bus_routes,
    'region_centers':points18,
    'neud_center':neud_center
}

In [9]:
G = indicators.assign_services_names_to_nodes(services,citygraph_n,citygraph,crs = local_crs)

railway_stops
fuel
no ports (((((((
local_aero
no international_aero (((((((
no oopt (((((((
no water_objects (((((((
no train_paths (((((((
bus_stops
capital
points
bus_routes
region_centers
neud_center


In [10]:
merged = indicators.indicator_area(G,inter,services,[city,polygons188,polygons18],local_crs)

100%|██████████| 2988/2988 [04:26<00:00, 11.19it/s]
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/metrics/indicators.py:713: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res = gpd.sjoin(at, polygons.to_crs(local_crs), how="left", predicate="within").groupby('index_right').median()
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/metrics/indicators.py:720: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  'index_right').median()
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/metrics/indicators.py:725: FutureWar

AttributeError: 'list' object has no attribute 'to_crs'

In [11]:
temp = indicators.indicator_territory(G,neud,local_crs,polygons18,points,points18,services)
temp

100%|██████████| 1/1 [00:00<00:00, 10.56it/s]


nearest_region_centers
nearest_settlement_centers


,name,to_region_admin_center,to_reg1,fuel_stations_accessibility,train_stops_accessibility,local_aero_accessibility,international_aero_accessibility,ports_accessibility,water_objects_accessibility,oopt_accessibility,...,number_of_international_aero,number_of_water_objects,number_of_oopt,number_of_bus_stops,density,train_path_length,number_of_bus_routes,to_nearest_settlement_center,to_nearest_mo_center,geometry
7910,None,49.587,5.376,0.000,0.000,70.714,None,None,0.000,0.554,...,0.000,674.000,0.000,116.000,1132.591,128.610,1.000,762.822,7518.282,"POLYGON ((37.86087 54.34286, 37.86265 54.34249..."


In [ ]:
merged.iloc[:,0:15]

,name,to_region_admin_center,to_reg1,fuel_stations_accessibility,train_stops_accessibility,local_aero_accessibility,international_aero_accessibility,ports_accessibility,water_objects_accessibility,oopt_accessibility,number_of_train_stops,number_of_fuel_stations,number_of_ports,number_of_local_aero,number_of_international_aero
7910,None,49.587,5.376,0.000,0.000,70.714,None,None,0.000,0.554,2.000,26.000,0.000,0.000,0.000


In [ ]:
merged.iloc[:,15:30]


,number_of_water_objects,number_of_oopt,number_of_bus_stops,density,train_path_length,number_of_bus_routes,to_nearest_settlement_center,to_nearest_mo_center,geometry
7910,674.000,0.000,116.000,1132.591,128.610,1.000,762.822,7518.282,"POLYGON ((37.86087 54.34286, 37.86265 54.34249..."
